## Installations

In [1]:
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=420be854c3285be806f760a1adfd4da93dd9ffd8f165081922f09c786451bcd0
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl


In [2]:
!pip install --no-index -U --find-links=/kaggle/input/tabm-tabular-dl-library tabm==0.0.1.dev0
!pip -q install /kaggle/input/pytorchtabnet/pytorch_tabnet-4.1.0-py3-none-any.whl

Looking in links: /kaggle/input/tabm-tabular-dl-library
Processing /kaggle/input/tabm-tabular-dl-library/tabm-0.0.1.dev0-py3-none-any.whl
Processing /kaggle/input/tabm-tabular-dl-library/rtdl_num_embeddings-0.0.11-py3-none-any.whl (from tabm==0.0.1.dev0)


In [3]:
!pip -q install /kaggle/input/tabpfn-v2/tabpfn-2.0.0-py3-none-any.whl

## Step 1 : Installations

In [4]:
import numpy as np
import pandas as pd
from scipy.stats import rankdata 

import sys
sys.path.append('/kaggle/input/tabm-tabular-dl-library')

import os
import tabm
import math
import torch
import random
import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
import rtdl_num_embeddings
import matplotlib.pyplot as plt
from typing import Optional, Tuple
from sklearn.model_selection import KFold
from scipy.stats import rankdata 
from colorama import Fore, Style
from typing import Optional, Tuple
from numpy.typing import ArrayLike
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from tabm_reference import Model, make_parameter_groups
from sklearn.preprocessing import OrdinalEncoder, QuantileTransformer
from pytorch_tabnet.tab_model import TabNetRegressor

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input, Embedding
from tensorflow.keras.layers import Concatenate, BatchNormalization
import tensorflow.keras.backend as K
from pytorch_tabnet.tab_model import TabNetRegressor
from tabpfn import TabPFNRegressor

## Step 2: Experiments Paths  to add in ensemble

In [5]:
experiments = [
"/kaggle/input/xgboost-exp-01",
"/kaggle/input/catboost-exp-01", 
"/kaggle/input/lgbm-exp-01",
"/kaggle/input/xgboost-exp-02",
"/kaggle/input/catboost-exp-02",
"/kaggle/input/lgbm-exp-03",
"/kaggle/input/catboost-exp-03",
"/kaggle/input/tabm-exp-01",
"/kaggle/input/nn-exp-01",
"/kaggle/input/tn-exp-01",
"/kaggle/input/tf-exp-01",
"/kaggle/input/svr-exp-01",
"/kaggle/input/abd-exp-01",
"/kaggle/input/catboost-exp-04",
"/kaggle/input/lgbm-exp-04",
"/kaggle/input/tabm-exp-02",
"/kaggle/input/ds-exp-01",
"/kaggle/input/nn-exp-02",
"/kaggle/input/nn-exp-04",
#"/kaggle/input/tabm-exp-03",
"/kaggle/input/catboost-exp-05",
"/kaggle/input/xgboost-exp-05",
"/kaggle/input/lgbm-exp-05",
"/kaggle/input/tn-exp-02",
#"/kaggle/input/ag-exp-01",
"/kaggle/input/vr-exp-01",
"/kaggle/input/tt-exp-01",
"/kaggle/input/en-exp-01",
"/kaggle/input/en-exp-02",
"/kaggle/input/nn-exp-05",
"/kaggle/input/rf-exp-05",
"/kaggle/input/mcts-exp-02",
"/kaggle/input/catboost-exp-06",
"/kaggle/input/xgboost-exp-06",
"/kaggle/input/lgbm-exp-06",
# "/kaggle/input/nn-exp-06",
# "/kaggle/input/xgboost-exp-07",
"/kaggle/input/ri-exp-01",
# "/kaggle/input/xgboost-exp-08",
# "/kaggle/input/catboost-exp-08",
# "/kaggle/input/lgbm-exp-08",
"/kaggle/input/xgboost-exp-09",
"/kaggle/input/prlnn-exp-01",
"/kaggle/input/ri-exp-06",
"/kaggle/input/xgboost-exp-10",
"/kaggle/input/lasso-exp-01",
"/kaggle/input/lir-exp-01"

# "/kaggle/input/suv-ran-exp-01" -> File not found
]

## Step 3 : Competition metric

In [6]:
import pandas as pd
import pandas.api.types
import numpy as np
from lifelines.utils import concordance_index

class ParticipantVisibleError(Exception):
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    del solution[row_id_column_name]
    del submission[row_id_column_name]
    
    event_label = 'efs'
    interval_label = 'efs_time'
    prediction_label = 'predictions'
    for col in submission.columns:
        if not pandas.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')
    # Merging solution and submission dfs on ID
    merged_df = pd.concat([solution, submission], axis=1)
    merged_df.reset_index(inplace=True)
    merged_df_race_dict = dict(merged_df.groupby(['race_group']).groups)
    metric_list = []

    for race in merged_df_race_dict.keys():
        # Retrieving values from y_test based on index
        indices = sorted(merged_df_race_dict[race])
        merged_df_race = merged_df.iloc[indices]
        # Calculate the concordance index
        c_index_race = concordance_index(
                        merged_df_race[interval_label],
                        -merged_df_race[prediction_label],
                        merged_df_race[event_label])
        metric_list.append(c_index_race)
    return float(np.mean(metric_list)-np.sqrt(np.var(metric_list)))

## Step 4: Find best model 

In [7]:
parquet_experiments = [
    "/kaggle/input/abd-exp-01",
    "/kaggle/input/lgbm-exp-04", 
    "/kaggle/input/catboost-exp-01", 
    "/kaggle/input/lgbm-exp-01", 
    "/kaggle/input/lgbm-exp-03", 
    "/kaggle/input/ds-exp-01",
    "/kaggle/input/nn-exp-02",
    "/kaggle/input/nn-exp-04",
    "/kaggle/input/tabm-exp-03",
    "/kaggle/input/catboost-exp-05",
    "/kaggle/input/xgboost-exp-05",
    "/kaggle/input/lgbm-exp-05",
    "/kaggle/input/tn-exp-02",
    #"/kaggle/input/ag-exp-01",
    "/kaggle/input/vr-exp-01",
    "/kaggle/input/tt-exp-01",
    "/kaggle/input/en-exp-01",
    "/kaggle/input/svr-exp-01",
    "/kaggle/input/en-exp-02",
    "/kaggle/input/nn-exp-05",
    "/kaggle/input/rf-exp-05",
    "/kaggle/input/mcts-exp-02",
    "/kaggle/input/catboost-exp-06",
    "/kaggle/input/xgboost-exp-06",
    "/kaggle/input/lgbm-exp-06",
    "/kaggle/input/nn-exp-06",
    "/kaggle/input/xgboost-exp-07",
    "/kaggle/input/ri-exp-01",
    "/kaggle/input/xgboost-exp-08",
    "/kaggle/input/catboost-exp-08",
    "/kaggle/input/lgbm-exp-08",
    "/kaggle/input/xgboost-exp-09",
    "/kaggle/input/prlnn-exp-01",
    "/kaggle/input/ri-exp-06",
    "/kaggle/input/xgboost-exp-10",
    "/kaggle/input/lasso-exp-01",
    "/kaggle/input/lir-exp-01"
]

In [8]:
def compute_metric_cindex(oof, exp_name):    
    y_true = oof[["ID","efs","efs_time","race_group"]].copy()
    y_pred = oof[["ID","predictions"]].copy()
    return score(y_true.copy(), y_pred.copy(), "ID"), oof

In [9]:
from tqdm import tqdm
import pandas as pd

# Initialize empty list to store individual prediction dataframes
dfs_to_merge = []

# Load first experiment to get the base structure
base_exp = experiments[0]
if base_exp in parquet_experiments:
    base_df = pd.read_parquet(base_exp + '/' + (base_exp.split('/')[-1]).replace('-','_') + '_oof.parquet')
else:
    base_df = pd.read_excel(base_exp + '/' + (base_exp.split('/')[-1]).replace('-','_') + '_oof.xlsx')

# Create base dataframe with ID and target variables
preds_df = base_df[["ID", "efs", "efs_time", "race_group"]].copy()

# Load and merge predictions from each experiment
for exp_name in tqdm(experiments):
    # Read the prediction file
    if exp_name in parquet_experiments:
        if "mcts-exp-02" in exp_name:
            curr_df = pd.read_parquet(exp_name + '/mcts_exp_01' + '_oof.parquet')
        else:
            curr_df = pd.read_parquet(exp_name + '/' + (exp_name.split('/')[-1]).replace('-','_') + '_oof.parquet')
    else:
        curr_df = pd.read_excel(exp_name + '/' + (exp_name.split('/')[-1]).replace('-','_') + '_oof.xlsx')
    
    # Create a temporary dataframe with ID and predictions
    temp_df = curr_df[["ID", "predictions"]].copy()
    temp_df = temp_df.rename(columns={"predictions": exp_name})
    
    # Merge with the main dataframe
    preds_df = preds_df.merge(temp_df, on="ID", how="left")

100%|██████████| 40/40 [03:54<00:00,  5.85s/it]


In [10]:
preds_df.shape, preds_df.columns 

((28800, 44),
 Index(['ID', 'efs', 'efs_time', 'race_group', '/kaggle/input/xgboost-exp-01',
        '/kaggle/input/catboost-exp-01', '/kaggle/input/lgbm-exp-01',
        '/kaggle/input/xgboost-exp-02', '/kaggle/input/catboost-exp-02',
        '/kaggle/input/lgbm-exp-03', '/kaggle/input/catboost-exp-03',
        '/kaggle/input/tabm-exp-01', '/kaggle/input/nn-exp-01',
        '/kaggle/input/tn-exp-01', '/kaggle/input/tf-exp-01',
        '/kaggle/input/svr-exp-01', '/kaggle/input/abd-exp-01',
        '/kaggle/input/catboost-exp-04', '/kaggle/input/lgbm-exp-04',
        '/kaggle/input/tabm-exp-02', '/kaggle/input/ds-exp-01',
        '/kaggle/input/nn-exp-02', '/kaggle/input/nn-exp-04',
        '/kaggle/input/catboost-exp-05', '/kaggle/input/xgboost-exp-05',
        '/kaggle/input/lgbm-exp-05', '/kaggle/input/tn-exp-02',
        '/kaggle/input/vr-exp-01', '/kaggle/input/tt-exp-01',
        '/kaggle/input/en-exp-01', '/kaggle/input/en-exp-02',
        '/kaggle/input/nn-exp-05', '/kaggle/inp

In [11]:
experiments

['/kaggle/input/xgboost-exp-01',
 '/kaggle/input/catboost-exp-01',
 '/kaggle/input/lgbm-exp-01',
 '/kaggle/input/xgboost-exp-02',
 '/kaggle/input/catboost-exp-02',
 '/kaggle/input/lgbm-exp-03',
 '/kaggle/input/catboost-exp-03',
 '/kaggle/input/tabm-exp-01',
 '/kaggle/input/nn-exp-01',
 '/kaggle/input/tn-exp-01',
 '/kaggle/input/tf-exp-01',
 '/kaggle/input/svr-exp-01',
 '/kaggle/input/abd-exp-01',
 '/kaggle/input/catboost-exp-04',
 '/kaggle/input/lgbm-exp-04',
 '/kaggle/input/tabm-exp-02',
 '/kaggle/input/ds-exp-01',
 '/kaggle/input/nn-exp-02',
 '/kaggle/input/nn-exp-04',
 '/kaggle/input/catboost-exp-05',
 '/kaggle/input/xgboost-exp-05',
 '/kaggle/input/lgbm-exp-05',
 '/kaggle/input/tn-exp-02',
 '/kaggle/input/vr-exp-01',
 '/kaggle/input/tt-exp-01',
 '/kaggle/input/en-exp-01',
 '/kaggle/input/en-exp-02',
 '/kaggle/input/nn-exp-05',
 '/kaggle/input/rf-exp-05',
 '/kaggle/input/mcts-exp-02',
 '/kaggle/input/catboost-exp-06',
 '/kaggle/input/xgboost-exp-06',
 '/kaggle/input/lgbm-exp-06',
 '

In [12]:
best_score    = 0
best_index    = -1
best_ensemble = 0

for k,name in enumerate(experiments):
    if "catboost-exp-05" in name:
        print("I am here")
    oof_pre = preds_df[["ID","efs","efs_time","race_group",name]]
    oof_pre = oof_pre.rename(columns={name: "predictions"})
    s, oof = compute_metric_cindex(oof_pre, name)
    if s > best_score:
        best_score    = s
        best_index    = name
        best_ensemble = oof
        
    print(f'C-index {s} {name}') 
print()
print(f'Best single model is {best_index} with C-Index = {best_score}')
experiments.remove(best_index)
first_best_index = best_index

C-index 0.6743662038463064 /kaggle/input/xgboost-exp-01
C-index 0.673797469068503 /kaggle/input/catboost-exp-01
C-index 0.6735659109490936 /kaggle/input/lgbm-exp-01
C-index 0.6719739976829171 /kaggle/input/xgboost-exp-02
C-index 0.6715438291634074 /kaggle/input/catboost-exp-02
C-index 0.6744113273004797 /kaggle/input/lgbm-exp-03
C-index 0.6750446351219317 /kaggle/input/catboost-exp-03
C-index 0.6698445972872783 /kaggle/input/tabm-exp-01
C-index 0.6672504731273489 /kaggle/input/nn-exp-01
C-index 0.6082304286386713 /kaggle/input/tn-exp-01
C-index 0.6609386133349129 /kaggle/input/tf-exp-01
C-index 0.6204664346764159 /kaggle/input/svr-exp-01
C-index 0.6775546112659996 /kaggle/input/abd-exp-01
C-index 0.6228381497451487 /kaggle/input/catboost-exp-04
C-index 0.6216961934710434 /kaggle/input/lgbm-exp-04
C-index 0.6777472728532595 /kaggle/input/tabm-exp-02
C-index 0.6427991982766068 /kaggle/input/ds-exp-01
C-index 0.6541867987232884 /kaggle/input/nn-exp-02
C-index 0.6794831551153862 /kaggle/in

In [13]:
experiments

['/kaggle/input/xgboost-exp-01',
 '/kaggle/input/catboost-exp-01',
 '/kaggle/input/lgbm-exp-01',
 '/kaggle/input/xgboost-exp-02',
 '/kaggle/input/catboost-exp-02',
 '/kaggle/input/lgbm-exp-03',
 '/kaggle/input/catboost-exp-03',
 '/kaggle/input/tabm-exp-01',
 '/kaggle/input/nn-exp-01',
 '/kaggle/input/tn-exp-01',
 '/kaggle/input/tf-exp-01',
 '/kaggle/input/svr-exp-01',
 '/kaggle/input/abd-exp-01',
 '/kaggle/input/catboost-exp-04',
 '/kaggle/input/lgbm-exp-04',
 '/kaggle/input/tabm-exp-02',
 '/kaggle/input/ds-exp-01',
 '/kaggle/input/nn-exp-02',
 '/kaggle/input/nn-exp-04',
 '/kaggle/input/catboost-exp-05',
 '/kaggle/input/xgboost-exp-05',
 '/kaggle/input/lgbm-exp-05',
 '/kaggle/input/tn-exp-02',
 '/kaggle/input/vr-exp-01',
 '/kaggle/input/tt-exp-01',
 '/kaggle/input/en-exp-01',
 '/kaggle/input/en-exp-02',
 '/kaggle/input/nn-exp-05',
 '/kaggle/input/rf-exp-05',
 '/kaggle/input/mcts-exp-02',
 '/kaggle/input/catboost-exp-06',
 '/kaggle/input/xgboost-exp-06',
 '/kaggle/input/lgbm-exp-06',
 '

## Step 5: Iterations for hill climbing

In [14]:
USE_NEGATIVE_WGT = True

In [15]:
indices        = [best_index]
old_best_score = best_score

In [16]:
# PREPARE/MOVE VARIABLES TO GPU FOR SPEED UP
best_ensemble = best_ensemble
start         = -0.50
if not USE_NEGATIVE_WGT: start = 0.01
ww            = np.arange(start,0.51,0.01) # GPU
nn            = len(ww)

In [17]:
# BEGIN HILL CLIMBING
models  = [best_index]
weights = []
metrics = [best_score]

In [18]:
models, metrics, ww

(['/kaggle/input/prlnn-exp-01'],
 [0.6817974276309455],
 array([-5.0000000e-01, -4.9000000e-01, -4.8000000e-01, -4.7000000e-01,
        -4.6000000e-01, -4.5000000e-01, -4.4000000e-01, -4.3000000e-01,
        -4.2000000e-01, -4.1000000e-01, -4.0000000e-01, -3.9000000e-01,
        -3.8000000e-01, -3.7000000e-01, -3.6000000e-01, -3.5000000e-01,
        -3.4000000e-01, -3.3000000e-01, -3.2000000e-01, -3.1000000e-01,
        -3.0000000e-01, -2.9000000e-01, -2.8000000e-01, -2.7000000e-01,
        -2.6000000e-01, -2.5000000e-01, -2.4000000e-01, -2.3000000e-01,
        -2.2000000e-01, -2.1000000e-01, -2.0000000e-01, -1.9000000e-01,
        -1.8000000e-01, -1.7000000e-01, -1.6000000e-01, -1.5000000e-01,
        -1.4000000e-01, -1.3000000e-01, -1.2000000e-01, -1.1000000e-01,
        -1.0000000e-01, -9.0000000e-02, -8.0000000e-02, -7.0000000e-02,
        -6.0000000e-02, -5.0000000e-02, -4.0000000e-02, -3.0000000e-02,
        -2.0000000e-02, -1.0000000e-02,  4.4408921e-16,  1.0000000e-02,
        

In [19]:
%%time

from tqdm import tqdm
import pandas as pd
import numpy as np
from typing import List, Dict, Tuple
import logging
import gc

def optimize_ensemble(experiments, initial_ensemble, weights_range, score_function, max_iterations = 100):
    
    # Initialize variables
    iteration  = 0
    best_score = score_function(
        initial_ensemble[["ID", "efs", "efs_time", "race_group"]].copy(),
        initial_ensemble[["ID", "predictions"]].copy(),
        "ID"
    )
    
    model_weights         = {}
    remaining_experiments = experiments.copy()
    best_ensemble         = initial_ensemble.copy()
    
    while remaining_experiments and iteration < max_iterations:
        print(f"{iteration}th iteration")
        iteration += 1
        best_iteration = {
            'index' : -1,
            'weight': 0,
            'score' : best_score
        }
        
        # Try each remaining model
        for model_path in remaining_experiments:
            try:
                #print(f"Iteration {iteration}: Trying model {model_path}")
                
                # Load model OOF predictions
                model_name = model_path.split("/")[-1].replace('-', '_')
                model_oof  = preds_df[model_path]
                
                # Try different weights
                for weight in tqdm(weights_range, desc=f"Testing weights for {model_name}"):
                    # Create potential ensemble
                    potential_ensemble = pd.DataFrame({
                        "ID": best_ensemble["ID"],
                        "predictions": (1 - weight) * rankdata(best_ensemble["predictions"]) + 
                                     weight * rankdata(model_oof)
                    })
                    
                    # Evaluate new ensemble
                    new_score = score_function(
                        preds_df[["ID", "efs", "efs_time", "race_group"]].copy(),
                        potential_ensemble.copy(),
                        "ID"
                    )
                    
                    # Update best if improved
                    if new_score > best_iteration['score']:
                        best_iteration.update({
                            'index' : model_path,
                            'weight': weight,
                            'score' : new_score
                        })
                
                # Clean up
                del model_oof
                gc.collect()
                
            except Exception as e:
                print(f"Error processing {model_path}: {str(e)}")
                continue
        
        # Check if we found an improvement
        if best_iteration['index'] == -1:
            print("No improvement found, stopping")
            print(best_iteration)
            break
            
        # Update ensemble with best model found
        best_score                             = best_iteration['score']
        model_weights[best_iteration['index']] = best_iteration['weight']
        remaining_experiments.remove(best_iteration['index'])
        
        print(
            f"Iteration {iteration}: Added {best_iteration['index']} "
            f"with weight {best_iteration['weight']:.4f}, "
            f"Score: {best_iteration['score']:.4f}"
        )
        
        # Update best ensemble for next iteration
        model_oof = preds_df[best_iteration['index']]
        best_ensemble["predictions"] = (
            (1 - best_iteration['weight']) * rankdata(best_ensemble["predictions"]) + 
            best_iteration['weight'] * rankdata(model_oof)
        )
        
    return model_weights, best_score


model_weights, best_score = optimize_ensemble(experiments, best_ensemble, ww, score)

0th iteration


Testing weights for lir_exp_01: 100%|██████████| 101/101 [00:38<00:00,  2.62it/s]


Iteration 1: Added /kaggle/input/catboost-exp-05 with weight 0.4800, Score: 0.6862
1th iteration


Testing weights for lir_exp_01: 100%|██████████| 101/101 [00:38<00:00,  2.61it/s]


Iteration 2: Added /kaggle/input/xgboost-exp-09 with weight 0.2700, Score: 0.6868
2th iteration


...

Iteration 26: Added /kaggle/input/tabm-exp-01 with weight 0.0000, Score: 0.6884
26th iteration


Testing weights for lir_exp_01: 100%|██████████| 101/101 [00:40<00:00,  2.49it/s]


Iteration 27: Added /kaggle/input/mcts-exp-02 with weight 0.0000, Score: 0.6884
27th iteration


Testing weights for lir_exp_01: 100%|██████████| 101/101 [00:38<00:00,  2.62it/s]

No improvement found, stopping
{'index': -1, 'weight': 0, 'score': 0.6884199045957253}
CPU times: user 7h 47min 14s, sys: 34.2 s, total: 7h 47min 48s
Wall time: 7h 47min 25s


In [20]:
# import cupy as cp
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import gc
# from scipy.stats import rankdata
# import time

# def optimize_ensemble_gpu(experiments, initial_ensemble, weights_range, score_function, max_iterations=100):
#     """
#     GPU-accelerated ensemble optimization using CuPy.
    
#     Args:
#         experiments: List of experiment paths to try
#         initial_ensemble: Initial ensemble predictions
#         weights_range: Range of weights to try
#         score_function: Function to evaluate ensemble quality
#         max_iterations: Maximum number of iterations
    
#     Returns:
#         model_weights: Dictionary of selected models and their weights
#         best_score: Best score achieved
#     """
#     # Initialize variables
#     iteration = 0
    
#     # Convert initial data to numpy for faster processing
#     ids = initial_ensemble["ID"].values
#     initial_preds = initial_ensemble["predictions"].values
    
#     # Pre-compute initial ensemble rank once (using numpy for initial ranking)
#     initial_ensemble_rank = rankdata(initial_preds)
    
#     # Move to GPU
#     cp_initial_ensemble_rank = cp.asarray(initial_ensemble_rank)
    
#     # Evaluate initial score
#     best_score = score_function(
#         initial_ensemble[["ID", "efs", "efs_time", "race_group"]].copy(),
#         initial_ensemble[["ID", "predictions"]].copy(),
#         "ID"
#     )
    
#     model_weights = {}
#     remaining_experiments = experiments.copy()
#     best_ensemble = initial_ensemble.copy()
    
#     # Pre-load all model predictions to avoid disk I/O in the loop
#     print("Pre-computing model ranks and moving to GPU...")
#     model_ranks_gpu = {}
#     for model_path in tqdm(experiments):
#         # Get model predictions and compute ranks
#         model_preds = preds_df[model_path].values
#         model_ranks = rankdata(model_preds)
        
#         # Move ranks to GPU
#         model_ranks_gpu[model_path] = cp.asarray(model_ranks)
    
#     # Clone initial ensemble rank for GPU operations
#     best_ensemble_rank_gpu = cp.copy(cp_initial_ensemble_rank)
    
#     # Create a GPU array for all weight values to test at once
#     cp_weights = cp.asarray(weights_range)
    
#     while remaining_experiments and iteration < max_iterations:
#         print(f"{iteration}th iteration, remaining models: {len(remaining_experiments)}")
#         iteration += 1
        
#         best_iteration = {
#             'index': -1,
#             'weight': 0,
#             'score': best_score
#         }
        
#         # Create batch scoring for all models and all weights
#         start_time = time.time()
        
#         for model_path in tqdm(remaining_experiments, desc="Testing models"):
#             # Get the model ranks from GPU memory
#             model_ranks_cp = model_ranks_gpu[model_path]
            
#             # Prepare GPU arrays for batch computation
#             all_scores = []
            
#             # Process weights in batches to avoid GPU memory issues
#             batch_size = min(len(weights_range), 25)  # Adjust based on GPU memory
            
#             for batch_start in range(0, len(weights_range), batch_size):
#                 batch_end = min(batch_start + batch_size, len(weights_range))
#                 batch_weights = cp_weights[batch_start:batch_end]
                
#                 # Reshape for broadcasting
#                 weights_reshaped = batch_weights.reshape(-1, 1)
                
#                 # GPU batch computation for all weights at once (for this model)
#                 # (1-w) * current_ensemble_rank + w * model_rank for each w in batch
#                 batch_results = cp.zeros((len(batch_weights), len(best_ensemble_rank_gpu)))
                
#                 for i, w in enumerate(batch_weights):
#                     batch_results[i] = (1 - w) * best_ensemble_rank_gpu + w * model_ranks_cp
                
#                 # Transfer back to CPU for scoring
#                 batch_results_np = cp.asnumpy(batch_results)
                
#                 # Evaluate all weights in this batch
#                 for i, w_idx in enumerate(range(batch_start, batch_end)):
#                     weight = weights_range[w_idx]
                    
#                     # Create ensemble for scoring
#                     potential_ensemble = pd.DataFrame({
#                         "ID": ids,
#                         "predictions": batch_results_np[i]
#                     })
                    
#                     # Evaluate the ensemble
#                     new_score = score_function(
#                         preds_df[["ID", "efs", "efs_time", "race_group"]].copy(),
#                         potential_ensemble,
#                         "ID"
#                     )
                    
#                     # Check if this is better
#                     if new_score > best_iteration['score']:
#                         best_iteration.update({
#                             'index': model_path,
#                             'weight': weight,
#                             'score': new_score
#                         })
            
#             # Free GPU memory
#             del batch_results
#             cp.get_default_memory_pool().free_all_blocks()
        
#         print(f"Iteration completed in {time.time() - start_time:.2f} seconds")
        
#         # Check if we found an improvement
#         if best_iteration['index'] == -1:
#             print("No improvement found, stopping")
#             break
            
#         # Update ensemble with best model found
#         best_score = best_iteration['score']
#         model_weights[best_iteration['index']] = best_iteration['weight']
#         remaining_experiments.remove(best_iteration['index'])
        
#         print(
#             f"Iteration {iteration}: Added {best_iteration['index']} "
#             f"with weight {best_iteration['weight']:.4f}, "
#             f"Score: {best_iteration['score']:.4f}"
#         )
        
#         # Update best ensemble ranks for next iteration using GPU computation
#         best_ensemble_rank_gpu = (
#             (1 - best_iteration['weight']) * best_ensemble_rank_gpu + 
#             best_iteration['weight'] * model_ranks_gpu[best_iteration['index']]
#         )
        
#         # Update actual predictions for final evaluation
#         best_ensemble["predictions"] = cp.asnumpy(best_ensemble_rank_gpu)
    
#     return model_weights, best_score


# def concordance_index_gpu(time_array, pred_array, event_array):
#     """
#     GPU-accelerated implementation of concordance index.
    
#     If your original concordance_index function is very complex,
#     you may need to adapt this further.
#     """
#     # Note: This is a placeholder implementation
#     # You'll need to adapt the specific concordance_index logic to GPU
#     # This would depend on your existing implementation
    
#     # Move data to GPU
#     time_gpu = cp.asarray(time_array)
#     pred_gpu = cp.asarray(pred_array)
#     event_gpu = cp.asarray(event_array)
    
#     # Compute concordance index on GPU
#     # ...GPU computation logic here...
    
#     # For now, fall back to your existing implementation
#     # Move data back to CPU
#     time_cpu = cp.asnumpy(time_gpu)
#     pred_cpu = cp.asnumpy(pred_gpu)
#     event_cpu = cp.asnumpy(event_gpu)
    
#     return concordance_index(time_cpu, pred_cpu, event_cpu)


# def score_gpu(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
#     """
#     GPU-accelerated scoring function.
    
#     This is a skeleton that you'd need to adapt based on your original score function.
#     """
#     sol = solution.copy()
#     sub = submission.copy()
    
#     del sol[row_id_column_name]
#     del sub[row_id_column_name]
    
#     event_label = 'efs'
#     interval_label = 'efs_time'
#     prediction_label = 'predictions'
    
#     # Merge solution and submission
#     merged_df = pd.concat([sol, sub], axis=1)
#     merged_df.reset_index(inplace=True)
    
#     # Group by race_group
#     race_groups = merged_df.groupby('race_group')
    
#     # Use GPU for faster calculations
#     c_indices = []
#     for race, group in race_groups:
#         # If you have a GPU version of concordance_index
#         c_index_race = concordance_index_gpu(
#             group[interval_label].values,
#             -group[prediction_label].values,
#             group[event_label].values
#         )
#         c_indices.append(c_index_race)
    
#     c_indices = np.array(c_indices)
#     return float(np.mean(c_indices) - np.sqrt(np.var(c_indices)))


# # Example usage
# if __name__ == "__main__":
#     try:
#         import cupy as cp
#         print("CuPy is available - Using GPU acceleration")
        
#         # Run GPU optimized version
#         model_weights, best_score = optimize_ensemble_gpu(
#             experiments, 
#             best_ensemble, 
#             weights_range=ww,
#             score_function=score_gpu,  # Or use score_gpu if you implement it
#             max_iterations=100
#         )
        
#         print(f"Best score: {best_score:.6f}")
#         print("Model weights:")
#         for model, weight in model_weights.items():
#             print(f"  {model}: {weight:.6f}")
            
#     except ImportError:
#         print("CuPy not available - falling back to CPU version")
#         # Run CPU version as fallback

In [21]:
model_weights, best_score

({'/kaggle/input/catboost-exp-05': 0.48000000000000087,
  '/kaggle/input/xgboost-exp-09': 0.2700000000000007,
  '/kaggle/input/nn-exp-04': 0.1600000000000006,
  '/kaggle/input/lasso-exp-01': -0.03999999999999959,
  '/kaggle/input/xgboost-exp-02': -0.08999999999999964,
  '/kaggle/input/svr-exp-01': -0.03999999999999959,
  '/kaggle/input/catboost-exp-01': 0.08000000000000052,
  '/kaggle/input/lgbm-exp-01': -0.09999999999999964,
  '/kaggle/input/rf-exp-05': -0.03999999999999959,
  '/kaggle/input/ds-exp-01': 0.05000000000000049,
  '/kaggle/input/catboost-exp-06': 0.0600000000000005,
  '/kaggle/input/ri-exp-06': -0.029999999999999583,
  '/kaggle/input/en-exp-02': 0.04000000000000048,
  '/kaggle/input/xgboost-exp-06': -0.07999999999999963,
  '/kaggle/input/tabm-exp-02': 0.05000000000000049,
  '/kaggle/input/nn-exp-01': -0.06999999999999962,
  '/kaggle/input/tn-exp-02': 0.03000000000000047,
  '/kaggle/input/nn-exp-05': -0.0499999999999996,
  '/kaggle/input/catboost-exp-03': -0.049999999999999

In [22]:
first_best_index, experiments

('/kaggle/input/prlnn-exp-01',
 ['/kaggle/input/xgboost-exp-01',
  '/kaggle/input/catboost-exp-01',
  '/kaggle/input/lgbm-exp-01',
  '/kaggle/input/xgboost-exp-02',
  '/kaggle/input/catboost-exp-02',
  '/kaggle/input/lgbm-exp-03',
  '/kaggle/input/catboost-exp-03',
  '/kaggle/input/tabm-exp-01',
  '/kaggle/input/nn-exp-01',
  '/kaggle/input/tn-exp-01',
  '/kaggle/input/tf-exp-01',
  '/kaggle/input/svr-exp-01',
  '/kaggle/input/abd-exp-01',
  '/kaggle/input/catboost-exp-04',
  '/kaggle/input/lgbm-exp-04',
  '/kaggle/input/tabm-exp-02',
  '/kaggle/input/ds-exp-01',
  '/kaggle/input/nn-exp-02',
  '/kaggle/input/nn-exp-04',
  '/kaggle/input/catboost-exp-05',
  '/kaggle/input/xgboost-exp-05',
  '/kaggle/input/lgbm-exp-05',
  '/kaggle/input/tn-exp-02',
  '/kaggle/input/vr-exp-01',
  '/kaggle/input/tt-exp-01',
  '/kaggle/input/en-exp-01',
  '/kaggle/input/en-exp-02',
  '/kaggle/input/nn-exp-05',
  '/kaggle/input/rf-exp-05',
  '/kaggle/input/mcts-exp-02',
  '/kaggle/input/catboost-exp-06',
  '

## Step 6: Inference on test

In [23]:
# class CFG:
#     folds = 10

# # https://www.kaggle.com/datasets/jsday96/mcts-tabm-models/data?select=TabMRegressor.py
# class TabMRegressor:
#     def __init__(
#         self,
#         arch_type: str        = 'tabm-mini',
#         backbone: dict        = {'type': 'MLP', 'n_blocks': 3, 'd_block': 512, 'dropout': 0.1},
#         d_embedding: int      = 64,  # Only used for 'tabm-mini'
#         bin_count: int        = 48,  # Only used for 'tabm-mini'
#         k: int                = 32,
#         learning_rate: float  = 1e-4,
#         weight_decay: float   = 1e-3,
#         clip_grad_norm: bool  = True,
#         max_epochs: int       = 100,
#         patience: int         = 15,
#         batch_size: int       = 32,
#         compile_model: bool   = False,
#         device: Optional[str] = 'cuda:0',
#         random_state: int     = 0,
#         verbose: bool         = True
#     ):
#         self.arch_type = arch_type
#         self.backbone = backbone
#         self.d_embedding = d_embedding
#         self.bin_count = bin_count
#         self.k = k
#         self.learning_rate = learning_rate
#         self.weight_decay = weight_decay
#         self.clip_grad_norm = clip_grad_norm
#         self.max_epochs = max_epochs
#         self.patience = patience
#         self.batch_size = batch_size
#         self.compile_model = compile_model
#         self.device = torch.device(device if device else ('cuda' if torch.cuda.is_available() else 'cpu'))
#         self.random_state = random_state
#         self.verbose = verbose

#     def fit(
#         self,
#         X: pd.DataFrame,
#         y: np.array,
#         eval_set: Tuple[pd.DataFrame, np.array]
#     ):
#         # PREPROCESS DATA.
#         X_cat_train, X_cont_train, cat_cardinalities, y_train = self._preprocess_data(X, y, training=True)
#         X_cat_val, X_cont_val, _, y_val = self._preprocess_data(eval_set[0], eval_set[1], training=False)

#         # CREATE MODEL & TRAINING ALGO.
#         bins = rtdl_num_embeddings.compute_bins(X_cont_train, n_bins=self.bin_count) if self.arch_type == 'tabm-mini' else None
#         self.model = Model(
#             n_num_features=X_cont_train.shape[1],
#             cat_cardinalities=cat_cardinalities,
#             n_classes=None,
#             backbone=self.backbone,
#             bins=bins,
#             num_embeddings=(
#                 None
#                 if bins is None
#                 else {
#                     'type': 'PiecewiseLinearEmbeddings',
#                     'd_embedding': self.d_embedding,
#                     'activation': True,
#                     'version': 'B',
#                 }
#             ),
#             arch_type=self.arch_type,
#             k=self.k,
#         ).to(self.device)
#         optimizer = torch.optim.AdamW(make_parameter_groups(self.model), lr=self.learning_rate, weight_decay=self.weight_decay)
#         if self.compile_model:
#             self.model = torch.compile(self.model)

#         loss_fn = torch.nn.MSELoss().to(self.device)
#         # TRAIN & TEST MODEL.
#         best = {
#             'epoch': -1,
#             'eval_loss': math.inf,
#             'model_state_dict': None,
#         }
#         remaining_patience = self.patience
#         epoch_size = math.ceil(len(X) / self.batch_size)


#         for epoch in range(self.max_epochs):
#             # TRAIN.
#             optimizer.zero_grad()
#             train_losses = []
#             progress_bar = torch.randperm(len(y_train), device=self.device).split(self.batch_size)
#             progress_bar = tqdm(progress_bar, desc=f'Epoch {epoch}', total=epoch_size) if self.verbose else progress_bar
#             for batch_idx in progress_bar:
#                 self.model.train()

#                 with torch.amp.autocast(device_type='cuda', dtype = torch.bfloat16):
#                     y_pred = self.model(
#                         X_cont_train[batch_idx],
#                         X_cat_train[batch_idx],
#                     ).squeeze(-1).float()

#                 loss = loss_fn(y_pred.flatten(0, 1), y_train[batch_idx].repeat_interleave(self.k))
#                 loss.backward()
#                 if self.clip_grad_norm:
#                     torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
#                 optimizer.step()

#                 train_losses.append(loss.item())


#              # EVALUATE.
#             self.model.eval()
#             val_losses = []
#             with torch.no_grad():
#                 for batch_idx in torch.arange(0, len(y_val), self.batch_size, device=self.device):
#                     y_pred = self.model(
#                         X_cont_val[batch_idx:batch_idx+self.batch_size],
#                         X_cat_val[batch_idx:batch_idx+self.batch_size],
#                     ).squeeze(-1).float()

#                     loss = loss_fn(y_pred.flatten(0, 1), y_val[batch_idx:batch_idx+self.batch_size].repeat_interleave(self.k))
#                     val_losses.append(loss.item())


#             # PRINT INFO.
#             mean_train_loss = np.mean(train_losses)
#             mean_val_loss = np.mean(val_losses)
#             if self.verbose:
#                 print(f'Epoch {epoch} | Train Loss: {mean_train_loss} | Val Loss: {mean_val_loss}')


#             # COMPARE TO BEST.
#             if mean_val_loss < best['eval_loss']:
#                 best['epoch'] = epoch
#                 best['eval_loss'] = mean_val_loss
#                 best['model_state_dict'] = self.model.state_dict()
#                 remaining_patience = self.patience
                
#                 if self.verbose:
#                     print('🌸 New best epoch! 🌸')
#             else:
#                 remaining_patience -= 1

#             # EARLY STOPPING.
#             if remaining_patience == 0:
#                 break

#             # RESTORE BEST MODEL.
#             self.model.load_state_dict(best['model_state_dict'])


#     def predict(
#         self,
#         X: pd.DataFrame,
#         batch_size: Optional[int] = 8096
#     ) -> np.ndarray:
#         # PREPROCESS DATA.
#         X_cat, X_cont, _, _ = self._preprocess_data(X, y=None, training=False)

#         # PREDICT.
#         self.model.eval()
#         y_pred = []
#         with torch.no_grad():
#             for batch_idx in torch.arange(0, len(X), batch_size, device=self.device):
#                 y_pred.append(
#                     self.model(
#                         X_cont[batch_idx:batch_idx+batch_size],
#                         X_cat[batch_idx:batch_idx+batch_size],
#                     ).squeeze(-1).float().cpu().numpy()
#                 )

#         y_pred = np.concatenate(y_pred)


#         # DENORMALIZE TARGETS.
#         y_pred = y_pred * self._target_std + self._target_mean


#         # COMPUTE ENSEMBLE MEAN.
#         y_pred = np.mean(y_pred, axis=1)

#         return y_pred


#     def _preprocess_data(self, X: pd.DataFrame, y: pd.Series, training: bool):
#         # PICK NON-CONSTANT COLUMNS.
#         if training:
#             self._non_constant_columns = X.columns[X.nunique() > 1]

#         X = X[self._non_constant_columns]

#         # SEPARATE CATEGORICAL & CONTINUOUS FEATURES.
#         categorical_features = [col for col in X.columns if X[col].dtype.name == 'object']
#         X_cat = X[categorical_features].to_numpy()
#         X_cont = X.drop(columns=categorical_features).to_numpy()

#         # ENCODE CATEGORICAL FEATURES.
#         cat_cardinalities = [X[col].nunique() for col in categorical_features]

#         if training:
#             self._categorical_encoders = [
#                 OrdinalEncoder()
#                 for _ in range(X_cat.shape[1])
#             ]
#         X_cat = np.concatenate([
#             encoder.fit_transform(X_cat[:, i:i+1])
#             for i, encoder in enumerate(self._categorical_encoders)
#         ], axis=1)

#         # NORMALIZE TARGETS.
#         if training:
#             self._target_mean = y.mean()
#             self._target_std = y.std()

#             y = (y - self._target_mean) / self._target_std


#         # SCALE CONTINUOUS FEATURES.
#         if training:
#             noise = (
#                 np.random.default_rng(0)
#                 .normal(0.0, 1e-5, X_cont.shape)
#                 .astype(X_cont.dtype)
#             )
#             self._cont_feature_preprocessor = QuantileTransformer(
#                 n_quantiles=max(min(len(X) // 30, 1000), 10),
#                 output_distribution='normal',
#                 subsample=10**9,
#             ).fit(X_cont + noise)

#         X_cont = self._cont_feature_preprocessor.transform(X_cont)


#         # CONVERT TO TENSORS.
#         X_cat = torch.tensor(X_cat, dtype=torch.long, device=self.device)
#         X_cont = torch.tensor(X_cont, dtype=torch.float32, device=self.device)

#         if y is not None:
#             y = torch.tensor(y, dtype=torch.float32, device=self.device)

#         return X_cat, X_cont, cat_cardinalities, y


In [24]:
# def get_tabm_features(data):
#     RMV = ["ID","efs","efs_time","y","fold"]
#     FEATURES = [c for c in data.columns if not c in RMV]
    
#     RMV              = ['ID']
#     X_test           = data.drop(RMV, axis=1)
#     y_pred           = data[['ID']]
    
#     #print("X_test shape:", X_test.shape, '\n')
    
#     cat_cols         = X_test.select_dtypes(include=['object']).columns.tolist()
#     num_cols         = X_test.select_dtypes(exclude=['object']).columns.tolist()
    
#     # Preprocessing categorical
#     imputer          = SimpleImputer(strategy='constant', fill_value='NAN')
#     X_test[cat_cols] = imputer.fit_transform(X_test[cat_cols])

#     # Preprocessing numerical
#     imputer          = SimpleImputer(strategy="median")
#     X_test[num_cols] = imputer.fit_transform(X_test[num_cols])

#     return X_test,FEATURES

In [25]:
# def prepare_features(model_path, train, test):

#     RMV = ["ID","efs","efs_time","y"]
#     FEATURES = [c for c in train.columns if not c in RMV]
#     #print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")
    

#     CATS = []
#     for c in FEATURES:
#         if train[c].dtype=="object":
#             CATS.append(c)
#             train[c] = train[c].fillna("NAN")
#             test[c]  = test[c].fillna("NAN")
#         elif "DeepTabels" in model_path or "tn" in model_path or "svr" in model_path:
#             train[c] = train[c].fillna(-1)
#             test[c]  = test[c].fillna(-1)
            
        
#     #print(f"In these features, there are {len(CATS)} CATEGORICAL FEATURES: {CATS}")
    
#     combined = pd.concat([train,test],axis=0,ignore_index=True)
#     #print("Combined data shape:", combined.shape )
    
#     # LABEL ENCODE CATEGORICAL FEATURES
#     #print("We LABEL ENCODE the CATEGORICAL FEATURES: ",end="")
#     for c in FEATURES:
    
#         # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
#         if c in CATS:
#             #print(f"{c}, ",end="")
#             combined[c],_ = combined[c].factorize()
#             combined[c]  -= combined[c].min()
#             combined[c]   = combined[c].astype("int32")
#             combined[c]   = combined[c].astype("category")
            
#         # REDUCE PRECISION OF NUMERICAL TO 32BIT TO SAVE MEMORY
#         else:
#             if combined[c].dtype =="float64":
#                 combined[c]      = combined[c].astype("float32")
#             if combined[c].dtype =="int64":
#                 combined[c]      = combined[c].astype("int32")
        
#     train = combined.iloc[:len(train)].copy()
#     test  = combined.iloc[len(train):].reset_index(drop=True).copy()
                
#     return train, test, FEATURES

In [26]:
# def get_nn_features(train, test):
    
#     CAT_SIZE = []
#     CAT_EMB  = []
#     NUMS     = []
#     CATS     = []

#     RMV = ["ID","efs","efs_time","y","fold"]
#     FEATURES = [c for c in train.columns if not c in RMV]
    
#     for c in FEATURES:
#         if train[c].dtype=="object":
#             train[c] = train[c].fillna("NAN")
#             test[c]  = test[c].fillna("NAN")
#             CATS.append(c)
#         elif not "age" in c:
#             train[c] = train[c].astype("str")
#             test[c]  = test[c].astype("str")
#             CATS.append(c)


#     combined = pd.concat([train,test],axis=0,ignore_index=True)
#     for c in FEATURES:
#         if c in CATS:
#             # LABEL ENCODE
#             combined[c],_ = combined[c].factorize()
#             combined[c] -= combined[c].min()
#             combined[c] = combined[c].astype("int32")
#             #combined[c] = combined[c].astype("category")

#             n = combined[c].nunique()
#             mn = combined[c].min()
#             mx = combined[c].max()
#             #print(f'{c} has ({n}) unique values')
    
#             CAT_SIZE.append(mx+1) 
#             CAT_EMB.append( int(np.ceil( np.sqrt(mx+1))) ) 
#         else:
#             if combined[c].dtype=="float64":
#                 combined[c] = combined[c].astype("float32")
#             if combined[c].dtype=="int64":
#                 combined[c] = combined[c].astype("int32")
                
#             m = combined[c].mean()
#             s = combined[c].std()
#             combined[c] = (combined[c]-m)/s
#             combined[c] = combined[c].fillna(0)
            
#             NUMS.append(c)

#     train = combined.iloc[:len(train)].copy()
#     test = combined.iloc[len(train):].reset_index(drop=True).copy()

#     return test[CATS], test[NUMS]

In [27]:
# def get_tf_features(train, test):
#     RMV = ["ID","efs","efs_time","y","y_na","fold"]
#     FEATURES = [c for c in train.columns if not c in RMV]


#     test                             = test.replace('Not done', 'missing')
#     test                             = test.replace('Not tested', 'missing')
    
#     test['na_count']                 = test.isna().sum(axis=1)
#     test['age_karnofsky']            = test['age_at_hct'] * test['karnofsky_score']
#     test['age_comorbidity']          = test['age_at_hct'] * test['comorbidity_score']
#     test['donor_recipient_age_diff'] = abs(test['donor_age'] - test['age_at_hct'])
#     test['hla_match_ratio']          = (test['hla_high_res_8'] + test['hla_low_res_8']) / 16
#     test['age_squared']              = test['age_at_hct'] ** 2
#     test['karnofsky_squared']        = test['karnofsky_score'] ** 2
#     test['16?']                      = np.where(test['age_at_hct']<=16,1,0)
    
#     FEATURES.extend(["na_count", "age_karnofsky", "age_comorbidity", "donor_recipient_age_diff", "hla_match_ratio", "age_squared", "karnofsky_squared", "16?"])

#     CATS = []
#     for c in FEATURES:
#         if test[c].dtype=="object":
#             CATS.append(c)
#             test[c] = test[c].fillna("missing")

#     for c in FEATURES:
#         # LABEL ENCODE CATEGORICAL AND CONVERT TO INT32 CATEGORY
#         if c in CATS:
#             #print(f"{c}, ",end="")
#             test[c],_ = test[c].factorize()
#             test[c]  -= test[c].min()
#             test[c]   = test[c].astype("int32")
#             test[c]   = test[c].astype("category")
#         else:
#             if test[c].dtype == "float64":
#                 test[c]      = test[c].astype("float32")
#             if test[c].dtype =="int64":
#                 test[c]      = test[c].astype("int32")
    
#     return test, FEATURES

In [28]:
# import pickle
# from tqdm import tqdm
# from sklearn.svm import SVR
# from sklearn.preprocessing import StandardScaler
# from sklearn.impute import KNNImputer

# imputer               = KNNImputer(n_neighbors=5, weights='uniform')

# scaler = StandardScaler()

# FOLDS = 10

# def inference(model_path, train, test_df):
    
#     path = model_path.split('/')[-1]
#     file = path.replace('-','_')

#     if "dt" not in model_path:
#         with open(f"/kaggle/input/{path}/{file}.pkl", 'rb') as f:
#             models = pickle.load(f)
            
#     print("All models are loaded successfully....!")

#     test_predictions = np.zeros(len(test_df))

#     for fold in tqdm(range(FOLDS)):
#         if "dt" in model_path:
#             # model = keras.models.load_model(model_path, custom_objects=custom_objects)
#             # train, test, FEATURES   = prepare_features(model_path, train, test_df)
#             # model                   = tf.keras.models.load_model(model_path+f"/model_fold_{fold}.keras", custom_objects=dt.__dict__, compile=)
#             # fold_preds              = model.predict(test.copy())
#             # fold_preds              = fold_preds.flatten()
#             pass
#         else:
#             model  = models[fold]
            
            
#         if "svr" in model_path:
#             if fold==0:
#                 train, test, FEATURES = prepare_features(model_path, train.copy(), test_df.copy())

#             # Handle missing values
#             train_imputed         = imputer.fit_transform(train[FEATURES].copy())
#             test_imputed          = imputer.transform(test[FEATURES])

#             # Convert back to DataFrame to maintain feature names
#             train_imputed         = pd.DataFrame(train_imputed, columns=FEATURES, index=train.index)
#             test_imputed          = pd.DataFrame(test_imputed, columns=FEATURES, index=test.index)
            
#             # Scale features
#             scaler.fit(train_imputed)
#             test_scaled           = scaler.transform(test_imputed)
            
#             fold_preds            = model.predict(test_scaled)

#         elif "tn" in model_path:
#             if fold == 0:
#                 train, test, FEATURES   = prepare_features(model_path, train, test_df)
                
#             fold_preds              = model.predict(test[FEATURES].values).flatten()
            
#         elif "nn" in model_path:
#             if fold == 0:
#                 X_cat, X_num      = get_nn_features(train, test_df.copy())
#             fold_preds        = model.predict([X_cat.values, X_num.values])
#             fold_preds        = fold_preds.flatten()

#         elif "tf" in model_path:
#             if fold == 0:
#                 test, FEATURES = get_tf_features(train.copy(),test_df.copy())
#                 fold_preds     = model.predict(test[FEATURES].copy())

#         elif "tabm" in model_path:
#             if fold == 0:
#                 test, FEATURES = get_tabm_features(test_df.copy())
#             fold_preds              = model.predict(test[FEATURES].copy())
            
#         else: 
#             if fold == 0:
#                 train, test, FEATURES   = prepare_features(model_path, train, test_df)
#             fold_preds              = model.predict(test[FEATURES].copy())
            
#         test_predictions += fold_preds
    
#     # Get the average predictionr
#     test_predictions /= FOLDS
        
#     return test_predictions

In [29]:
# train_df = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
# test_df  = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")

In [30]:
# first_best_index

In [31]:
# initial_test_preds = inference(first_best_index, train_df, test_df)
# initial_test_preds

In [32]:
# model_weights

In [33]:
# test_preds = []
# for model, weight in model_weights.items():
#     print(f"Using model : {model}")
#     test_df            = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
    
#     test_preds         = inference(model, train_df.copy(), test_df)
#     test_preds         = (1-weight) * rankdata(initial_test_preds) + weight * rankdata(test_preds)
#     initial_test_preds = test_preds

In [34]:
# train_df               = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
# test_df                = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
# test_preds             = np.zeros(len(test_df))


# preds_dict = {}
# for model, weight in model_weights.items():
#     print(f"exp name : {model}\n")
#     test_df                = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
#     test_preds             = inference(model, train_df.copy(), test_df.copy())
#     test_preds             = (1-weight) * rankdata(initial_test_preds) + weight * rankdata(test_preds)
#     initial_test_preds     = test_preds

In [35]:
# test_preds

In [36]:
# potential_ensemble                = pd.DataFrame()
# potential_ensemble["predictions"] = test_preds
# potential_ensemble["ID"]          = test_df["ID"]
# new_score                         = score(test_df[["ID","efs","efs_time","race_group"]].copy(), potential_ensemble.copy(), "ID")
# new_score

In [37]:
# best_score

## Step 7: Create submission file

In [38]:
# sub            = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")
# sub.prediction = test_preds
# sub.to_csv("submission.csv",index=False)
# print("Sub shape:",sub.shape)
# sub.head()